In [2]:
import torch
import torch.distributions as dist
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib macosx

In [3]:
S = torch.tensor([[[i+j*10+k*100 for j in range(5)]for i in range(6)] for k in range(3)], dtype=torch.float32)
D = torch.tensor([[i*j*1e-3 for j in range(1,7)]for i in range(1,8)], dtype=torch.float32)

print(S.shape)
print(D.shape)
# tensor matriz multiplication
result = torch.matmul(D,S)
print(result.shape)     
# Output:

torch.Size([3, 6, 5])
torch.Size([7, 6])
torch.Size([3, 7, 5])


In [4]:
m = dist.exponential.Exponential(D).sample()
print(m)
print(D[0])


tensor([[1665.2599,  176.1533,   28.0372,   61.8147,   84.5176,  244.7314],
        [ 773.2141,   87.9213,  363.5797,   86.6103,   94.9722,  290.6422],
        [ 661.2239,  233.0332,   47.2806,    6.9104,   16.8565,    4.5003],
        [ 224.0149,    5.1182,  131.6726,   27.8145,   14.7444,   83.2639],
        [  99.8607,  129.4865,  145.1152,    3.0843,   14.8113,   35.2229],
        [  87.1474,  123.1670,  106.0655,   25.4124,   63.2059,   20.0016],
        [ 106.4156,   44.1319,   12.5505,   86.8919,   10.7864,   24.2725]])
tensor([0.0010, 0.0020, 0.0030, 0.0040, 0.0050, 0.0060])


In [5]:
ns = 1000
tde = dist.exponential.Exponential(D[0]).sample((ns,)).view(-1)
spe = torch.zeros_like(tde)
for i, alpha in enumerate(D[0]):
    t =  torch.tensor(stats.expon.rvs(scale=1/alpha,size=ns),dtype=torch.float32)
    spe[i*ns:(i+1)*ns] = t
# show a histogram of the samples
fig, ax = plt.subplots(2)
ax[0].hist(tde, bins=20, density=True, alpha=0.6, color='b')
ax[1].hist(spe, bins=20, density=True, alpha=0.6, color='g')

(array([2.00351852e-03, 6.66368563e-04, 2.82985959e-04, 1.39010646e-04,
        7.61248778e-05, 4.19238458e-05, 3.58559352e-05, 1.87553971e-05,
        1.32391038e-05, 9.92933590e-06, 3.30977863e-06, 4.96466394e-06,
        5.51629327e-06, 4.41304175e-06, 5.51629327e-07, 1.10325865e-06,
        5.51629327e-07, 0.00000000e+00, 5.51630218e-07, 1.10325865e-06]),
 array([3.44225345e-03, 3.02138580e+02, 6.04273682e+02, 9.06408813e+02,
        1.20854395e+03, 1.51067908e+03, 1.81281421e+03, 2.11494922e+03,
        2.41708447e+03, 2.71921973e+03, 3.02135474e+03, 3.32348975e+03,
        3.62562500e+03, 3.92776025e+03, 4.22989502e+03, 4.53203027e+03,
        4.83416553e+03, 5.13630078e+03, 5.43843604e+03, 5.74057080e+03,
        6.04270605e+03]),
 <BarContainer object of 20 artists>)

In [ ]:
global_inter = []

class ParticleGrid:
    
    def __init__(self, N: int = 1000, b= 100.0, k: float = 10.0, jl: float = 200, jr: float = 200, init_func=None):
        self.N = N
        self.particles = torch.zeros((N, 4), dtype=torch.float32)
        self.init_func = init_func
        if self.init_func is None:
            self.init_func = lambda x: (torch.cos(2*torch.pi*x)+1)/2.0
        self.b = b
        self.k = k
        self.jl = jl
        self.jr = jr
        self.epsilon = 1e-8
        self.count = 0
        self.initialize(f)

    def initialize(self, f):
        x = torch.linspace(-1., 1., self.N)
        y = f(x)
        r = torch.distributions.uniform.Uniform(0, 1).sample((self.N,))
        self.grid = torch.zeros(self.N, dtype=torch.int)
        index = y>r
        self.grid[index]+=1
        self.cloks_parameters = torch.zeros((self.N, 4), dtype=torch.float32)
        self.clocks = torch.zeros((self.N, 4), dtype=torch.float32)
        self.update_clocks()
    
    def update_clocks(self, interval=slice(None)):
        self.count += 1
        self.cloks_parameters[interval, 0] = self.grid[interval,]*self.b  + self.epsilon
        self.cloks_parameters[interval, 1] = self.grid[interval,]*self.k  + self.epsilon
        self.cloks_parameters[interval, 2] = self.grid[interval,]*self.jl + self.epsilon
        self.cloks_parameters[interval, 3] = self.grid[interval,]*self.jr + self.epsilon
        self.clocks[interval,:] = dist.exponential.Exponential(self.cloks_parameters[interval,:]).sample()
        

    def run(self, steps: int = 100):
        for _ in range(steps):
            next_site = torch.argmin(self.clocks)
            next_site = torch.unravel_index(next_site, self.clocks.shape)
            pass_time = self.clocks[next_site].item()
            self.clocks -= pass_time
            self.clocks[next_site] = torch.inf
            self.update_site(next_site)

    def update_site(self, next_site):
        clock_type = next_site[1]
        if clock_type == 0:
            self.grid[next_site[0]] += 1
            self.update_clocks(next_site[0])
        elif clock_type == 1:
            if self.grid[next_site[0]] > 0:
                self.grid[next_site[0]] -= 1
                self.update_clocks(next_site[0])
        elif clock_type == 2:
            if self.grid[next_site[0]] > 0:
                nn = (next_site[0] + 1) % self.N
                self.grid[next_site[0]] -= 1
                self.grid[nn] += 1
                self.update_clocks((next_site[0].item(), nn.item()))
        elif clock_type == 3:
            if self.grid[next_site[0]] > 0:
                nn = (next_site[0] - 1) % self.N
                self.grid[next_site[0]] -= 1
                self.grid[nn] += 1
                self.update_clocks((next_site[0].item(), nn.item()))
        

In [32]:
#plot grid
def plot_grid(grid):
    plt.plot(grid, marker='o', linestyle='', markersize=2)
    plt.xlabel('Particle index')
    plt.ylabel('Number of particles')
    plt.title('Particle grid')
    plt.show()



In [33]:
pg = ParticleGrid(2048)
pg.run(2**15)
plot_grid(pg.grid)



In [34]:
pg = ParticleGrid(2048)
plot_grid(pg.grid)
for _ in range(4):
    pg.run(2**10)
    plot_grid(pg.grid)


In [42]:
# animation of the particle grid
pg = ParticleGrid(1024, b=1, k=.004, jl=40, jr=40)
fig, ax = plt.subplots()

line, = ax.plot(pg.grid, marker='o', linestyle='')
ax.set_ylim(-0.1,1.1)

ax.set_xlabel('Particle index')
ax.set_ylabel('Number of particles')
ax.set_title('Particle grid')
def update(frame):
    pg.run(2**5)
    line.set_ydata(pg.grid/pg.grid.max())  # Normalize the grid for better visualization
    ax.set_xlabel(f'Particle index {pg.count}')
    ax.set_title(f'Particle grid \n B={pg.b}, K={pg.k}, JL={pg.jl}, JR={pg.jr} \n Max #particles:{pg.grid.max()} - Total particles: {pg.grid.sum()}')

    return line,

ani = animation.FuncAnimation(fig=fig, func=update, frames=100, interval=2, blit=False)
fig.show()